In [ ]:
# | default_exp classes.DomoPage


In [ ]:
# | exporti
from nbdev import show_doc
from fastcore.basics import patch_to
from dataclasses import dataclass, field

import asyncio
import httpx

import domolibrary.client.Logger as lg
import domolibrary.client.DomoError as de
import domolibrary.client.DomoAuth as dmda
import domolibrary.classes.DomoPage_Content as dmpg_c
import domolibrary.routes.page as page_routes

import domolibrary.utils.DictDot as util_dd
import domolibrary.utils.chunk_execution as ce


# DomoPage


In [ ]:
# | export
@dataclass
class DomoPage:
    id: int
    title: str = None
    parent_page_id: int = None
    top_page_id: int = None
    auth: dmda.DomoAuth = field(default=None, repr=False)
    is_locked: bool = None

    parent_hierarchy: [dict] = None
    parent_page: dict = None  # DomoPage
    top_page: dict = None  # DomoPage

    collections: list = field(default_factory=list)

    owners: list = field(default_factory=list)
    cards: list = field(default_factory=list)
    children: list = field(default_factory=list)
    flat_children: list = None

    layout: dmpg_c.PageLayout = None

    def display_url(self):
        return f"https://{self.auth.domo_instance}.domo.com/page/{self.id}"

    async def _get_domo_owners_from_dd(self, owners: util_dd.DictDot):

        import domolibrary.classes.DomoUser as dmu
        import domolibrary.classes.DomoGroup as dmg

        tasks = []
        owner_group_ls = [
            owner.id for owner in owners if owner.type == "GROUP" and owner.id]

        if len(owner_group_ls) > 0:
            [tasks.append(dmg.DomoGroup.get_by_id(group_id=group_id,
                                                  auth=self.auth))
                for group_id in owner_group_ls]

        owner_user_ls = [
            owner.id for owner in owners if owner.type == "USER" and owner.id]

        if len(owner_user_ls) > 0:
            tasks.append(dmu.DomoUsers.by_id(
                user_ids=owner_user_ls, only_allow_one=False, auth=self.auth))

        owner_ce = await ce.gather_with_concurrency(n=60, *tasks)

        res = []
        for owner in owner_ce:
            if isinstance(owner, list):
                [res.append(member) for member in owner]
            else:
                res.append(owner)

        return res


In [ ]:
# | exporti
@patch_to(DomoPage, cls_method=True)
async def _from_adminsummary(cls, page_obj, auth: dmda.DomoAuth):
    import domolibrary.classes.DomoCard as dmc

    dd = page_obj
    if isinstance(page_obj, dict):
        dd = util_dd.DictDot(page_obj)

    pg = cls(
        id=int(dd.id or dd.pageId),
        title=dd.title or dd.pageTitle,
        parent_page_id=int(dd.parentPageId) if dd.parentPageId else None,
        top_page_id=int(dd.topPageId) if dd.topPageId else None,
        collections=dd.collections,
        is_locked=dd.locked,
        auth=auth,
    )

    if dd.page and dd.page.owners and len(dd.page.owners) > 0:
        pg.owners = await pg._get_domo_owners_from_dd(dd.page.owners)

    if dd.cards and len(dd.cards) > 0:
        pg.cards = await ce.gather_with_concurrency(n=60,
                                                    *[dmc.DomoCard.get_from_id(id=card.id, auth=auth) for card in dd.cards]
                                                    )

    return pg


In [ ]:
# | exporti

@patch_to(DomoPage, cls_method=True)
async def _from_bootstrap(cls: DomoPage, page_obj, auth: dmda.DomoAuth = None):
    dd = page_obj
    if isinstance(page_obj, dict):
        dd = util_dd.DictDot(page_obj)

    pg = cls(id=int(dd.id), title=dd.title, auth=auth)

    if isinstance(dd.owners, list) and len(dd.owners) > 0:
        pg.owners = await pg._get_domo_owners_from_dd(dd.owners)

    if isinstance(dd.children, list) and len(dd.children) > 0:
        pg.children = await ce.gather_with_concurrency(n=60,
                                                       *[
                                                           cls._from_bootstrap(
                                                               page_obj=child_dd, auth=auth)
                                                           for child_dd in dd.children
                                                           if child_dd.type == "page"
                                                       ]
                                                       )

        [print(other_dd)
         for other_dd in dd.children if other_dd.type != "page"]

    return pg


# Domo Pages


In [ ]:
# | export
@dataclass
class DomoPages:

    @classmethod
    async def get_pages(
        cls,
        auth=dmda.DomoAuth,
        return_raw: bool = False,
        debug_loop: bool = False,
        debug_api: bool = False,
        session: httpx.AsyncClient = None,
    ):
        is_close_session = False if session else True

        session = session or httpx.AsyncClient()

        try:
            res = await page_routes.get_pages_adminsummary(
                auth=auth, debug_loop=False, debug_api=False, session=session
            )

            if return_raw:
                return res

            if not res.is_success:
                raise Exception("unable to retrieve pages")

            return await ce.gather_with_concurrency(n=60,
                                                    *[
                                                        DomoPage._from_adminsummary(
                                                            page_obj, auth=auth)
                                                        for page_obj in res.response
                                                    ]
                                                    )

        finally:
            if is_close_session:
                await session.aclose()


#### sample implementation of get_pages


In [ ]:
from pprint import pprint
import os

token_auth = dmda.DomoTokenAuth(
    domo_instance="domo-community",
    domo_access_token=os.environ["DOMO_DOJO_ACCESS_TOKEN"],
)

domo_pages = await DomoPages.get_pages(auth=token_auth, return_raw=False)
domo_pages[0:5]


[DomoPage(id=1316566624, title='20210623_TRAINING_DomoStats Activity Log App', parent_page_id=127044793, top_page_id=522373865, is_locked=False, parent_hierarchy=None, parent_page=None, top_page=None, collections=None, owners=[], cards=[], children=[], flat_children=None, layout=None),
 DomoPage(id=384424178, title='75th Percentile Test', parent_page_id=None, top_page_id=None, is_locked=False, parent_hierarchy=None, parent_page=None, top_page=None, collections=None, owners=[], cards=[], children=[], flat_children=None, layout=None),
 DomoPage(id=1917664953, title='A', parent_page_id=1603030697, top_page_id=2054898013, is_locked=False, parent_hierarchy=None, parent_page=None, top_page=None, collections=None, owners=[], cards=[], children=[], flat_children=None, layout=None),
 DomoPage(id=1113451501, title='Aditya Jain Dev', parent_page_id=None, top_page_id=None, is_locked=True, parent_hierarchy=None, parent_page=None, top_page=None, collections=None, owners=[], cards=[], children=[], fl

# DomoPage Cont

## get_by_id


In [ ]:
# | exporti
@patch_to(DomoPage, cls_method=True)
async def _from_content_stacks_v3(cls: DomoPage, page_obj, auth: dmda.DomoAuth = None):
    # import domolibrary.classes.DomoCard as dc

    dd = page_obj
    if isinstance(page_obj, dict):
        dd = util_dd.DictDot(page_obj)

    pg = cls(
        id=int(dd.id),
        title=dd.title,
        parent_page_id=int(
            dd.page.parentPageId) if dd.page.parentPageId else None,
        collections=dd.collections,
        auth=auth,
    )

    if hasattr(dd, "pageLayoutV4") and dd.pageLayoutV4 is not None:
        pg.layout = dmpg_c.PageLayout._from_json(dd=dd.pageLayoutV4)

    if dd.page.owners and len(dd.page.owners) > 0:
        pg.owners = await pg._get_domo_owners_from_dd(dd.page.owners)

    # if dd.cards and len(dd.cards) > 0:
    #     pg.cards = await asyncio.gather(
    #         *[dc.DomoCard.get_from_id(id=card.id, auth=auth) for card in dd.cards])

    return pg


class DomoPage_GetRecursive(de.DomoError):
    def __init__(self,  include_recursive_children, include_recursive_parents, page_id, domo_instance, function_name, parent_class):
        super().__init__(
            domo_instance=domo_instance,
            function_name=function_name,
            parent_class=parent_class,
            message=f"error retrieving {page_id} can only trace parents OR children recursively but not both. include_recursive_children : {include_recursive_children}, include_recursive_parents: {include_recursive_parents}"
        )


@patch_to(DomoPage, cls_method=True)
async def get_by_id(
    cls: DomoPage,
    page_id: str,
    auth: dmda.DomoAuth,
    return_raw: bool = False,
    debug_api: bool = False,
    include_layout: bool = False,

    # if True, will drill down to all the Children.  Set to False to prevent calculating children
    include_recursive_children: bool = True,
    include_recursive_parents: bool = False,
):

    # can only trace upstream or downstream but not both
    if include_recursive_children and include_recursive_parents:
        traceback_details = lg.get_traceback()

        raise DomoPage_GetRecursive(include_recursive_children=include_recursive_children,
                                    include_recursive_parents=include_recursive_parents,
                                    page_id=page_id,
                                    domo_instance=auth.domo_instance,
                                    function_name=traceback_details.function_name, parent_class=cls.__name__)

    res = await page_routes.get_page_by_id(
        auth=auth, page_id=page_id, debug_api=debug_api, include_layout=include_layout
    )

    if return_raw:
        return res

    if not res.is_success:
        return None

    pg = await cls._from_content_stacks_v3(page_obj=res.response, auth=auth)

    if pg.parent_page_id and include_recursive_parents:
        pg.parent_page = await cls.get_by_id(auth=auth, page_id=pg.parent_page_id,
                                             include_recursive_parents=include_recursive_parents,
                                             include_recursive_children=False
                                             )

        if pg.parent_page:
            pg.parent_hierarchy = pg.get_parent_hierarchy()

            pg.top_page = pg.parent_hierarchy[-1]['page']
            pg.top_page_id = pg.parent_hierarchy[-1]['page'].id

    if include_recursive_children:
        await pg.get_children(include_recursive_children=include_recursive_children,
                              )
        pg.flat_children = pg.flatten_children()

    return pg


@patch_to(DomoPage)
def get_parent_hierarchy(self: DomoPage, path=None, hierarchy=0, results=None):
    results = results or []

    path = path or self.title

    results.append(
        {"hierarchy": hierarchy,
         "path":  path,
         "page": self})

    if self.parent_page:
        path = f"{path} > {self.parent_page.title}"
        self.parent_page.get_parent_hierarchy(path, hierarchy+1, results)

    return results


@patch_to(DomoPage)
async def get_children(self: DomoPage,
                       include_recursive_children: bool = False):

    all_pages = await DomoPages.get_pages(auth=self.auth)

    self.children = await ce.gather_with_concurrency(
        n=10,
        *[
            DomoPage.get_by_id(
                page_id=page.id,
                auth=self.auth,
                include_recursive_children=include_recursive_children,
                include_recursive_parents=False
            )
            for page in all_pages
            if page.parent_page_id == self.id
        ]
    )

    return self.children


@patch_to(DomoPage)
def flatten_children(self: DomoPage, path=None, hierarchy=0, results=None):
    results = results or []

    path = f"{path} > {self.title}" if path else self.title

    results.append(
        {"hierarchy": hierarchy,
            "path":  path,
            "page": self})

    if self.children:
        [child.flatten_children(path, hierarchy+1, results)
         for child in self.children]

    return results


#### sample implementations of get_by_id

retrieve page without get_children_recursion


In [ ]:
token_auth = dmda.DomoTokenAuth(
    domo_instance="domo-community",
    domo_access_token=os.environ["DOMO_DOJO_ACCESS_TOKEN"],
)

page_id = 109096720

await DomoPage.get_by_id(
    page_id=page_id, auth=token_auth,
    return_raw=False,
    include_layout=True,
    include_recursive_children=False
)


DomoPage(id=109096720, title='Domo IDEA Exchange', parent_page_id=None, top_page_id=None, is_locked=None, parent_hierarchy=None, parent_page=None, top_page=None, collections=[], owners=[DomoUser(id='1893952720', title=None, department=None, display_name='Jae Wilson1', email_address='jae@onyxreporting.com', role_id=810756122, avatar_key=None, phone_number=None, web_landing_page=None, web_mobile_landing_page=None, employee_id=None, employee_number=None, hire_date=None, reports_to=None, publisher_domain=None, subscriber_domain=None, virtual_user_id=None)], cards=[], children=[], flat_children=None, layout=PageLayout(id=391912007, page_id='109096720', is_print_friendly=True, is_enabled=True, is_dynamic=False, has_page_breaks=False, content=[PageLayoutContent(accept_date_filter=True, accept_filters=True, accept_segments=True, card_id=721561037, card_urn='721561037', compact_interaction_default=True, content_key=0, fit_to_frame=False, has_summary=False, hide_border=False, hide_description=Tr

##### with parent_page and top_page


In [ ]:
token_auth = dmda.DomoTokenAuth(
    domo_instance="domo-community",
    domo_access_token=os.environ["DOMO_DOJO_ACCESS_TOKEN"],
)

page_id = 717329130

domo_page = await DomoPage.get_by_id(
    page_id=page_id, auth=token_auth,
    return_raw=False,
    include_layout=True,
    include_recursive_parents=True,
    include_recursive_children=False
)

print(domo_page.top_page)
print(domo_page.parent_page)
domo_page.parent_hierarchy

DomoPage(id=109096720, title='Domo IDEA Exchange', parent_page_id=None, top_page_id=None, is_locked=None, parent_hierarchy=None, parent_page=None, top_page=None, collections=[], owners=[DomoUser(id='1893952720', title=None, department=None, display_name='Jae Wilson1', email_address='jae@onyxreporting.com', role_id=810756122, avatar_key=None, phone_number=None, web_landing_page=None, web_mobile_landing_page=None, employee_id=None, employee_number=None, hire_date=None, reports_to=None, publisher_domain=None, subscriber_domain=None, virtual_user_id=None)], cards=[], children=[], flat_children=None, layout=None)
DomoPage(id=531423275, title='Domo APIs', parent_page_id=109096720, top_page_id=109096720, is_locked=None, parent_hierarchy=[{'hierarchy': 0, 'path': 'Domo APIs', 'page': ...}, {'hierarchy': 1, 'path': 'Domo APIs > Domo IDEA Exchange', 'page': DomoPage(id=109096720, title='Domo IDEA Exchange', parent_page_id=None, top_page_id=None, is_locked=None, parent_hierarchy=None, parent_page

[{'hierarchy': 0,
  'path': 'CLI',
  'page': DomoPage(id=717329130, title='CLI', parent_page_id=531423275, top_page_id=109096720, is_locked=None, parent_hierarchy=[{'hierarchy': 0, 'path': 'CLI', 'page': ...}, {'hierarchy': 1, 'path': 'CLI > Domo APIs', 'page': DomoPage(id=531423275, title='Domo APIs', parent_page_id=109096720, top_page_id=109096720, is_locked=None, parent_hierarchy=[{'hierarchy': 0, 'path': 'Domo APIs', 'page': ...}, {'hierarchy': 1, 'path': 'Domo APIs > Domo IDEA Exchange', 'page': DomoPage(id=109096720, title='Domo IDEA Exchange', parent_page_id=None, top_page_id=None, is_locked=None, parent_hierarchy=None, parent_page=None, top_page=None, collections=[], owners=[DomoUser(id='1893952720', title=None, department=None, display_name='Jae Wilson1', email_address='jae@onyxreporting.com', role_id=810756122, avatar_key=None, phone_number=None, web_landing_page=None, web_mobile_landing_page=None, employee_id=None, employee_number=None, hire_date=None, reports_to=None, publi

with get_children_recursion


In [ ]:
from pprint import pprint
import os

token_auth = dmda.DomoTokenAuth(
    domo_instance="domo-community",
    domo_access_token=os.environ["DOMO_DOJO_ACCESS_TOKEN"],
)

page_id = 109096720

domo_page = await DomoPage.get_by_id(
    page_id=page_id, auth=token_auth, return_raw=False,
    include_recursive_children=True,
    include_recursive_parents=False
)

[   f"{fc['hierarchy']} - {fc['path']} : {fc['page'].title}" for fc in domo_page.flat_children]


['0 - Domo IDEA Exchange : Domo IDEA Exchange',
 '1 - Domo IDEA Exchange > Beast Modes : Beast Modes',
 '1 - Domo IDEA Exchange > Crypto Use-Case : Crypto Use-Case',
 '1 - Domo IDEA Exchange > Custom Apps : Custom Apps',
 '1 - Domo IDEA Exchange > Data Stories : Data Stories',
 '1 - Domo IDEA Exchange > Domo APIs : Domo APIs',
 '2 - Domo IDEA Exchange > Domo APIs > CLI : CLI',
 '2 - Domo IDEA Exchange > Domo APIs > Developer Portal : Developer Portal',
 '2 - Domo IDEA Exchange > Domo APIs > SDKs : SDKs',
 '1 - Domo IDEA Exchange > Forecasting / Inline Editor App : Forecasting / Inline Editor App',
 '1 - Domo IDEA Exchange > Text Analytics : Text Analytics']

In [ ]:
# | export
@patch_to(DomoPage)
async def get_accesslist(
    self,
    auth: dmda.DomoAuth = None,
    is_expand_users: bool = False,
    return_raw: bool = False,
    debug_api: bool = False,
):
    auth = auth or self.auth

    res = await page_routes.get_page_access_list(
        auth=auth,
        is_expand_users=is_expand_users,
        page_id=self.id,
        debug_api=debug_api
    )

    if return_raw:
        return res

    if not res.is_success:
        raise Exception("error getting access list")

    import domolibrary.classes.DomoUser as dmu
    import domolibrary.classes.DomoGroup as dmg

    tasks_cn = await ce.gather_with_concurrency(n=60,
                                                * [dmu.DomoUsers.by_id(
                                                    user_ids=[
                                                        user.get("id") for user in res.response.get("users")],
                                                    only_allow_one=False,
                                                    auth=auth)],
                                                *[
                                                    dmg.DomoGroup.get_by_id(group_id=group.get("id"),
                                                                            auth=auth)
                                                    for group in res.response.get("groups")
                                                ]
                                                )
    return [*tasks_cn[0], tasks_cn[1]]


#### sample get_accesslist


In [ ]:
from pprint import pprint
import os

token_auth = dmda.DomoTokenAuth(
    domo_instance="domo-community",
    domo_access_token=os.environ["DOMO_DOJO_ACCESS_TOKEN"],
)

page_id = 1761849366
domo_page = DomoPage(id=page_id, auth=token_auth)

res = await domo_page.get_accesslist(is_expand_users=True, return_raw=False, debug_api=False)
res


[DomoUser(id='1216550715', title=None, department=None, display_name='8:26 - go to sleep', email_address='test4@domo.com', role_id=2, avatar_key=None, phone_number=None, web_landing_page=None, web_mobile_landing_page=None, employee_id=None, employee_number=None, hire_date=None, reports_to=None, publisher_domain=None, subscriber_domain=None, virtual_user_id=None),
 DomoUser(id='1345408759', title=None, department=None, display_name='Alexis Lorenz (DataMaven)', email_address='DataMaven42@gmail.com', role_id=1, avatar_key=None, phone_number=None, web_landing_page=None, web_mobile_landing_page=None, employee_id=None, employee_number=None, hire_date=None, reports_to=None, publisher_domain=None, subscriber_domain=None, virtual_user_id=None),
 DomoUser(id='1416802348', title=None, department=None, display_name='Dean', email_address='dean.pangelinan@bluemoondigital.co', role_id=2, avatar_key=None, phone_number=None, web_landing_page=None, web_mobile_landing_page=None, employee_id=None, employe

In [ ]:
# | exporti
@patch_to(DomoPage)
async def share(self: DomoPage,
                auth: dmda.DomoAuth = None,
                domo_users: list = None,  # DomoUsers to share page with,
                domo_groups: list = None,  # DomoGroups to share page with
                message: str = None,  # message for automated email
                debug_api: bool = False, session: httpx.AsyncClient = None):

    import domolibrary.routes.datacenter as datacenter_routes

    if domo_groups:
        domo_groups = domo_groups if isinstance(
            domo_groups, list) else [domo_groups]
    if domo_users:
        domo_users = domo_users if isinstance(
            domo_users, list) else [domo_users]

    res = await datacenter_routes.share_resource(
        auth=auth or self.auth,
        resource_ids=[self.id],
        resource_type=datacenter_routes.ShareResource_Enum.PAGE,
        group_ids=[group.id for group in domo_groups] if domo_groups else None,
        user_ids=[user.id for user in domo_users] if domo_users else None,
        message=message,
        debug_api=debug_api, session=session,
    )

    return res


#### sample share_page


In [ ]:
import os
import domolibrary.client.DomoAuth as dmda
import domolibrary.classes.DomoGroup as dmg


token_auth = dmda.DomoTokenAuth(
    domo_instance="domo-community",
    domo_access_token=os.environ["DOMO_DOJO_ACCESS_TOKEN"],
)

domo_group = await dmg.DomoGroup.get_by_id(group_id=1324037627, auth=token_auth)

domo_page = DomoPage(id=30507758, auth=token_auth)

await domo_page.share(
    auth=token_auth,
    domo_groups=[domo_group],
    message=None,
    debug_api=False,
)


ResponseGetData(status=200, response='page 30507758 successfully shared with 1324037627', is_success=True, parent_class=None, traceback_details=TracebackDetails(function_name='share', file_name='/tmp/ipykernel_39941/4074100456.py', function_trail='<module> -> share', traceback_stack=[<FrameSummary file /tmp/ipykernel_39941/3059594826.py, line 15 in <module>>, <FrameSummary file /tmp/ipykernel_39941/4074100456.py, line 19 in share>], parent_class=None))

In [ ]:
# | exporti

@patch_to(DomoPage, cls_method=True)
async def get_cards(cls,
                    auth: dmda.DomoAuth,
                    page_id, debug_api: bool = False,
                    session: httpx.AsyncClient = None):

    import domolibrary.classes.DomoCard as dc

    res = await page_routes.get_page_definition(auth=auth, page_id=page_id, debug_api=debug_api, session=session)

    if res.status != 200:
        raise Exception(
            f"unable to retrieve page definition for {page_id} in {auth.domo_instance}")

    if len(res.response.get('cards')) == 0:
        return []

    return await ce.gather_with_concurrency(n=60, *[dc.DomoCard.get_by_id(card_id=card['id'],
                                                                          auth=auth) for card in res.response.get('cards')])


@patch_to(DomoPage, cls_method=True)
async def get_datasets(cls,
                       auth: dmda.DomoAuth,
                       page_id,
                       debug_api: bool = False,
                       session: httpx.AsyncClient = None):

    import domolibrary.classes.DomoDataset as dmds

    res = await page_routes.get_page_definition(auth=auth, page_id=page_id,
                                                debug_api=debug_api, session=session)

    if res.status != 200:
        raise Exception(
            f"unable to retrieve datasets for page {page_id} in {auth.domo_instance}")

    if len(res.response.get('cards')) == 0:
        return []

    return await ce.gather_with_concurrency(n=60, *[dmds.DomoDataset.get_from_id(dataset_id=ds.get('dataSourceId'), auth=auth) for card in res.response.get('cards') for ds in card.get('datasources')])


In [ ]:

import os
import domolibrary.client.DomoAuth as dmda

token_auth = dmda.DomoTokenAuth(
    domo_instance="domo-community",
    domo_access_token=os.environ["DOMO_DOJO_ACCESS_TOKEN"],
)

res = await DomoPage.get_cards(page_id=30507758, auth=token_auth)

res[0:5]


[DomoCard(id=16688637, title='Slack.png', description='Slack.png', type='document', urn='16688637', chart_type=None, dataset_id=None, certification=DictDot(state='NOT_CERTIFIED', adminCertified=False), owners=[DomoUser(id='587894148', title='Executive Analytics', department='BT Partners', display_name='Bryan Van Kampen', email_address='bryan@bvankampen.net', role_id=1, avatar_key='09/6CF6C801EEF440968C88ECC9824D66', phone_number='6304647034', web_landing_page=None, web_mobile_landing_page=None, employee_id=None, employee_number=None, hire_date=1588827600000, reports_to=None, publisher_domain=None, subscriber_domain=None, virtual_user_id=None)]),
 DomoCard(id=1581742516, title='Dashboard Engagement Applet', description=None, type='domoapp', urn='1581742516', chart_type='domoapp', dataset_id='bcfb536a-f4d8-4d1b-8d68-c6b2bf116ae8', certification=DictDot(state='NOT_CERTIFIED', adminCertified=False), owners=[DomoUser(id='68216396', title=None, department='Owner', display_name='Elliott Leona

In [ ]:

import os
import domolibrary.client.DomoAuth as dmda

token_auth = dmda.DomoTokenAuth(
    domo_instance="domo-community",
    domo_access_token=os.environ["DOMO_DOJO_ACCESS_TOKEN"],
)

res = await DomoPage.get_datasets(page_id=30507758, auth=token_auth)

res[0:5]


[DomoDataset(id='bcfb536a-f4d8-4d1b-8d68-c6b2bf116ae8', display_type='webform', data_provider_type='webform', name='Budget 2022', description=None, row_count=12, column_count=2, stream_id=858, owner={'id': '68216396', 'name': 'Elliott Leonard', 'type': 'USER', 'group': False}, formula={}, schema=DomoDataset_Schema(dataset=..., columns=[]), tags=DomoDataset_Tags(dataset=..., tag_ls=[]), certification=None, PDP=<domolibrary.classes.DomoPDP.Dataset_PDP_Policies object>),
 DomoDataset(id='8a721f95-d78b-4f8d-be8b-e06e913c4a5a', display_type='webform', data_provider_type='webform', name='Profit 2022', description=None, row_count=13, column_count=2, stream_id=860, owner={'id': '68216396', 'name': 'Elliott Leonard', 'type': 'USER', 'group': False}, formula={}, schema=DomoDataset_Schema(dataset=..., columns=[]), tags=DomoDataset_Tags(dataset=..., tag_ls=[]), certification=None, PDP=<domolibrary.classes.DomoPDP.Dataset_PDP_Policies object>),
 DomoDataset(id='586d93d4-a31c-4457-8cb1-5d5417d63398'

In [ ]:
# | exporti
from datetime import datetime
from utils import convert


@patch_to(DomoPage, cls_method=True)
async def update_layout(
    cls, auth: dmda.DomoAuth, body: dict, layout_id: str, debug_api: bool = False
):
    datetime_now = datetime.now()
    start_time_epoch = convert.convert_datetime_to_epoch_millisecond(
        datetime_now)

    res_writelock = await page_routes.put_writelock(
        auth=auth,
        layout_id=layout_id,
        user_id=auth.user_id,
        epoch_time=start_time_epoch,
    )
    if res_writelock.status == 200:
        res = await page_routes.update_page_layout(
            auth=auth, body=body, layout_id=layout_id, debug_api=debug_api
        )

        if not res.is_success:
            return False

        res_writelock = await page_routes.delete_writelock(
            auth=auth, layout_id=layout_id
        )
        if res_writelock.status != 200:
            return False

    else:
        return False

    return True


# Sample update page layout by background color


In [ ]:
from pprint import pprint
import os

token_auth = dmda.DomoTokenAuth(
    domo_instance="domo-community",
    domo_access_token=os.environ["DOMO_DOJO_ACCESS_TOKEN"],
)

page_id = 1761849366
domo_page = await DomoPage.get_by_id(
    page_id=page_id, auth=token_auth, return_raw=False, include_layout=True
)

body = domo_page.layout.get_body()

if not hasattr(body, "background"):
    new_background_body = dmpg_c.PageLayout.generate_new_background_body()
    body["background"] = new_background_body

body["background"]["selectedColor"] = "#FF0000"
auth = dmda.DomoFullAuth(
    domo_instance="domo-community",
    domo_password=os.environ["DOJO_PASSWORD"],
    domo_username=os.environ["DOMO_USERNAME"],
)

await DomoPage.update_layout(auth=auth, body=body, layout_id=domo_page.layout.id)


True

In [ ]:
# | hide
import nbdev

nbdev.nbdev_export()
